# Modular cloud native development in Python and Jupyter - demo using padded resizing of 2D images

## A. Set the working directory

In [2]:
import os
print('Current working dir: ', os.getcwd())

Current working dir:  /workspace/best_practices_1


Change directory if it is not in your desired repo/directory

In [ ]:
os.chdir('/workspace/best_practices_1')

### 2. Import Libraries and helper functions

In [7]:
#%%writefile 'src/resize/resize_pad.py'
import cv2
import yaml
from imutils import paths
import argparse
import pickle
import os
#from src.utils import save_object

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

### 3. Edit Configuration file with inputs needed for the task

In [ ]:
config_file = 'src/resize/params_single.yaml'
print(config_file)
with open(config_file) as conf_file:
    config = yaml.safe_load(conf_file)
print(config)

In [ ]:
config['data']['data_dir'] = 'data/demo'
config['data']['output_dir'] = 'output/demo/resize'
config['data']['pkl_dir'] = 'pickles/demo/resize'
config['desired_size'] = 224
config['output'] = {'format':'png'}
#del config['data']['input_csv']
#print(config)
#data = main(config)

In [ ]:
with open('src/resize/params_single.yaml', 'w') as outfile:
    yaml.dump(config, outfile, default_flow_style=False)

### 3. Define functions needed to accomplish task on a single datum

In [ ]:
#%%writefile -a 'src/resize/resize_pad.py'
def resize_pad(input_image,desired_size,toRGB): 
        im = input_image
        if toRGB:
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        
        old_size = im.shape[:2] # old_size is in (height, width) format
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        # new_size should be in (width, height) format

        im = cv2.resize(im, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        color = [0, 0, 0]
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
            value=color)
        return new_im

### 4. Define main function to load and write data for the task

In [ ]:
def main(config): 
    # Get input
    imfiles = [f for f in os.listdir(config['data']["data_dir"]) if not f.startswith('.')]
    data = []
    
    # Process
    for f in imfiles:
        im = cv2.imread(os.path.join(config['data']['data_dir'],f))
        new_im = resize_pad(im,config['desired_size'],config['toRGB'])
        data.append(new_im)

    # Create Output
    if config['output']['format'] == 'png':      
        outdir = config['data']['output_dir']
        os.makedirs(outdir,exist_ok=True)
        for f,d in zip(imfiles,data):
            cv2.imwrite(os.path.join(outdir,f), d)
    elif config['output']['format'] == 'pkl':
        outdir = config['data']['pkl_dir']
        os.makedirs(outdir,exist_ok=True)
        save_object(data,os.path.join(outdir,'resized_output.pkl'))
    
    print('resize_pad complete')
    return data, imfiles

### 5. Debug and iterate until code works 

In [ ]:
data, imfiles = main(config)

### 6. If code works add command "%%writefile 'path_to_file.py'" to section 2 cells, and  "%%writefile -a 'path_to_file.py'" to cells in section 3,4 and 6 [cell below] and execute the cells.

Code below is needed for python module to take config file as input and execute the .py file

In [ ]:
%%writefile -a 'src/resize/resize_pad.py'
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Configuration file to run resize_pad()')
    parser.add_argument('--config', dest='config', required=True)
    args = parser.parse_args()
    with open(args.config) as conf_file:
        config = yaml.safe_load(conf_file)
    
    print('Output variables data and infiles')
    data, infiles = main(config)

### 7. Test if module was created correctly by running it in terminal or in bash mode:

In [ ]:
# Test if it works in module format
!python3 src/resize/resize_pad.py --config src/resize/params_single.yaml

### 8. Create requirements file for packaging a task specific Docker Container

In [ ]:
import pigar

In [ ]:
!pigar -p 'src/resize/requirements.txt' -P 'src/resize'

### 9. [Optional] Write a Dockerfile to create a container image for the app/task

In [ ]:
%%writefile -a 'src/resize/resize_pad.py'
FROM Python

COPY resize/requirements.txt /opt/requriements.txt

RUN pip install --no-cache-dir 